In [1]:
%%time
# Run precompiled pin tool with student's executable
import subprocess


cacheSize = "200000"
numLinesOut = "10000000"
studentExecutable = "./executable"


csvFileName = "pin" #not sure if this should be input by student or just set to a default since CSV will be behind the scenes

pinExecutable = "/setup/pintool/pin"
cacheDumpSOPath = "/setup/pintool/source/tools/ManualExamples/obj-intel64/cache_dump.so"
csvFile = csvFileName + "_mem.out"

args = [pinExecutable,"-t", cacheDumpSOPath , "-o",csvFileName, "-c", cacheSize, '-m', numLinesOut,"--",studentExecutable]
completed = subprocess.run(args,capture_output=True)
print(completed.stdout.decode('utf-8'))

# display csv file line count
#completed = subprocess.run(["wc", csvFile], capture_output=True) # num lines, num words, num bytes
#print(completed.stdout.decode('utf-8'))


CPU times: user 2.36 ms, sys: 6.13 ms, total: 8.5 ms
Wall time: 10.8 s


In [7]:
%%time
# Run pre compiled converter
cacheOutFile = "cache_out.h5"

args = ['./convert', csvFile, cacheOutFile]
completed = subprocess.run(args,capture_output=True)
print(completed.stdout.decode('utf-8'))


CPU times: user 2.39 ms, sys: 9.07 ms, total: 11.5 ms
Wall time: 11.8 s


In [3]:
import ipywidgets as widgets
from ipywidgets import Checkbox, VBox, HBox
import vaex
import vaex.jupyter
import numpy as np
import pylab as plt

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


#Enumerations
READ_HIT = b'R'
READ_MISS = b'S'
WRITE_HIT = b'W'
WRITE_MISS = b'X'


df = vaex.open(cacheOutFile)
df['index'] = np.arange(0, df.ADDRESS.count())

#Set up name-tag mapping
namesFromFile = ["Hardcoded Tag 1", "2"]
tagsFromFile = np.unique(df.tag.values).tolist()
numTags = len(namesFromFile)



#Define all selections here
df.select(True, name='structures')
df.select(True, name='rw')
df.select(True, name='hm')
df.select(True, name='total')


#Initialize selections
for tag in tagsFromFile:
    df.select(df.tag == tag, mode='or')

    
#Handle all boolean combinations of independent checkboxes here
def combineSelections():
    df.select("structures", mode='replace', name='total')
    df.select("rw", mode='and', name='total')
    df.select("hm", mode='and', name='total')
    
    
    
def updateGraph(change):
    if(change.name == 'value'):
        
        name = change.owner.description
        tag = tagsFromFile[namesFromFile.index(name)]

        if(change.new == True):
            df.select(df.tag == tag, mode='or', name='structures')
            combineSelections()
        else:
            df.select(df.tag == tag, mode='subtract', name='structures')
            combineSelections()
    df.select('total')
        
        
        
def updateReadWrite(change):
    if(change.name == 'value'):
        
        name = change.owner.description
        
        if(name == "Reads"):
            targetHit = READ_HIT
            targetMiss = READ_MISS
        else:
            targetHit = WRITE_HIT
            targetMiss = WRITE_MISS
        
        if(change.new == True):
            df.select(df.WRITE == targetHit, mode='or', name='rw')
            df.select(df.WRITE == targetMiss, mode='or', name='rw')
            combineSelections()
        else:
            df.select(df.WRITE == targetHit, mode='subtract', name='rw')
            df.select(df.WRITE == targetMiss, mode='subtract', name='rw')
            combineSelections()
    df.select('total')
            
        
            
            
def updateHitMiss(change):
    if(change.name == 'value'):
        
        name = change.owner.description
        
        if(name == "Hits"):
            targetRead = READ_HIT
            targetWrite = WRITE_HIT
        else:
            targetRead = READ_MISS
            targetWrite = WRITE_MISS
        
        if(change.new == True):
            df.select(df.WRITE == targetRead, mode='or', name='hm')
            df.select(df.WRITE == targetWrite, mode='or', name='hm')
            combineSelections()
        else:
            df.select(df.WRITE == targetRead, mode='subtract', name='hm')
            df.select(df.WRITE == targetWrite, mode='subtract', name='hm')
            combineSelections()
    df.select('total')
        

        
        
tagButtons = [Checkbox(description=name, value=True, disabled=False, indent=False) for name in namesFromFile]

rwButtons = [Checkbox(description="Reads", value=True, disabled=False, indent=False),
             Checkbox(description="Writes", value=True, disabled=False, indent=False)]

hmButtons = [Checkbox(description="Hits", value=True, disabled=False, indent=False),
             Checkbox(description="Misses", value=True, disabled=False, indent=False)]
    
    
for i in range(numTags):
    tagButtons[i].observe(updateGraph)
    
for button in rwButtons:
    button.observe(updateReadWrite)
    
for button in hmButtons:
    button.observe(updateHitMiss)
    
checks = HBox([VBox(tagButtons), VBox(rwButtons), VBox(hmButtons)])

df.plot_widget(df.index, df.ADDRESS, selection=[True], backend='bqplot', tool_select=True)


display(checks)

PlotTemplate(components={'main-widget': VBox(children=(VBox(children=(Figure(axes=[Axis(color='#666', grid_col…

In [4]:
# Display data 
df

#,ADDRESS,WRITE,tag,index
<i style='opacity: 0.6'>0</i>,140403457728528,b'S',0,0
<i style='opacity: 0.6'>1</i>,140403457728528,b'W',0,1
<i style='opacity: 0.6'>2</i>,140403457728532,b'S',0,2
<i style='opacity: 0.6'>3</i>,140403457728532,b'W',0,3
<i style='opacity: 0.6'>4</i>,140403457728536,b'S',0,4
...,...,...,...,...
"<i style='opacity: 0.6'>9,999,995</i>",140403459728320,b'W',0,9999995
"<i style='opacity: 0.6'>9,999,996</i>",140403459728324,b'R',0,9999996
"<i style='opacity: 0.6'>9,999,997</i>",140403459728324,b'W',0,9999997
"<i style='opacity: 0.6'>9,999,998</i>",140403459728328,b'R',0,9999998


In [40]:
hex(140475668321224)

'0x7fc30a4f53c8'

In [6]:
df.close_files()

In [8]:
%%time
# TO CONVERT cache_accesses.out
# Run pre compiled converter
import subprocess

csvAccessFile = "cache_accesses.out"
cacheAccessesFile = "accesses.h5"

args = ['./convertAccesses', csvAccessFile, cacheAccessesFile]
completed = subprocess.run(args,capture_output=True)
print(completed.stdout.decode('utf-8'))


CPU times: user 6.48 ms, sys: 7.26 ms, total: 13.7 ms
Wall time: 185 ms


In [9]:
import vaex
df_access = vaex.open(cacheAccessesFile)
df_access

#,ACCESS
<i style='opacity: 0.6'>0</i>,140403459728328
<i style='opacity: 0.6'>1</i>,140733306391768
<i style='opacity: 0.6'>2</i>,140733306391760
<i style='opacity: 0.6'>3</i>,140733306391744
<i style='opacity: 0.6'>4</i>,140733306391824
...,...
"<i style='opacity: 0.6'>199,995</i>",140403458928620
"<i style='opacity: 0.6'>199,996</i>",140403458928616
"<i style='opacity: 0.6'>199,997</i>",140403458928612
"<i style='opacity: 0.6'>199,998</i>",140403458928608


In [10]:
df_access.close_files()

In [7]:
import ipywidgets as widgets
import inspect
from ipywidgets import Checkbox, VBox
import vaex
import vaex.jupyter
import numpy as np
import pylab as plt



#df = vaex.open("../hdf5_pinIO/test9/mill_access.h5")
df['index'] = np.arange(0, df.ADDRESS.count())
np.unique(df.tag.values).tolist()
arr = np.unique(df.WRITE.values).tolist()
df[:200010]
df.plot_widget(df.tag, df.ADDRESS, selection=[True], backend='bqplot', tool_select=True)



PlotTemplate(components={'main-widget': VBox(children=(VBox(children=(Figure(axes=[Axis(color='#666', grid_col…

Plot2dDefault(w=None, what='count(*)', x='tag', y='ADDRESS', z=None)